In [ ]:
import numpy as np
import pandas as pd
import math
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import (
    HBox,
    VBox,
    Output,
    FloatSlider,
    Button,
    ToggleButton,
    Layout,
    Dropdown,
)


In [ ]:
df = pd.read_csv('test-dataset/higgs_boson.csv').drop(['EventId','Weight'],axis=1)
columns=list(df.columns)
df

In [ ]:
dfs = df[df['Label']=='s']
dfb = df[df['Label']=='b']
i=0
#fig, ax = plt.subplots(10,10)
fig  = plt.figure(figsize=(25,25))
for var in df.columns:
    plt.subplot(8,8,i+1)
    plt.hist(dfb[var],label='background',alpha=0.4,color='red')
    plt.hist(dfs[var],label='signal',alpha=0.4,color='blue')
    i = i +  1
    plt.xlabel(var)
    plt.legend()

In [ ]:
DER_mass_vis = FloatSlider(min=df[columns[0]].min(),max=df[columns[0]].max(),step=1, description=columns[0], continuous_update=True,layout=widgets.Layout(min_width="70%"),)
DER_mass_jet_jet = FloatSlider(min=df[columns[1]].min(),max=df[columns[1]].max(),step=1, description=columns[1], continuous_update=True,layout=widgets.Layout(min_width="70%"),)
lin_log = widgets.Checkbox(value=False, description="Log")

In [ ]:
def significance(sig,bkg):
    if bkg == 0:
        print('background too low')
        return 0
    Z = sig/math.sqrt(sig+bkg)
    return Z

In [ ]:
def _cuts(lin_log,DER_mass_vis,DER_mass_jet_jet):

    plt.figure(figsize=(7,7))

    df2 = df[df['DER_mass_vis'] > DER_mass_vis]
    df2 = df2[df2['DER_mass_jet_jet'] > DER_mass_jet_jet]
        
    df_s = df2[df2['Label'] == 's']    
    df_b = df2[df2['Label'] == 'b']
    n_sig = df_s.value_counts().sum()
    n_bkg = df_b.value_counts().sum()

    
    print('s/sqrt(s+b) ', significance(n_sig,n_bkg))

    plt.hist(df_s['DER_mass_jet_jet'], label='signal', bins=50, color=['blue'],alpha=0.6)
    plt.hist(df_b['DER_mass_jet_jet'], label='background', bins=50, color=['red'],alpha=0.6)
    plt.legend()

    if lin_log and df2.shape[0]!=0:
        print("h")
        plt.yscale("log")
    else:
        plt.yscale("linear")

out = widgets.interactive_output(_cuts, {
                    'DER_mass_vis': DER_mass_vis,
                    'DER_mass_jet_jet': DER_mass_jet_jet,
                    'lin_log': lin_log,
                                        })

#{'lep1_pt': lep1_pt, 'lep2_pt': lep2_pt, 'Qnet': Qnet, 'mass_low': mass_low, 'mass_high':mass_high, 'lin_log':lin_log})
ui = VBox(
    [
        VBox([DER_mass_vis, DER_mass_jet_jet, lin_log]),
    ]
)

HBox([out, ui])
